# PageRank Implementation

## Methods To Implement Page Rank


We can compute PageRank in three ways
   - Random surfer
   - Probability spreading
   - Eigen vector of Transition Probability Matrix

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd